## Procesado de Ficheros III
---

### 3. Series temporales
---
En el ejemplo precedente (calidad del vino), cada muestra (_sample_) o registro de la tabla era independiente de las otras.

En nuestro siguiente ejemplo, vamos a lidiar con series de datos que tienen una componente temporal, lo que induce cierta ordenación en los datos de entrada.

Para nuestra práctica, vamos a utilizar un _dataset_ de alquiler de bicicletas en la ciudad de Washington D.C. en los años 2011-2012. En dichos datos de entrada, cada fila se corresponde con una hora de datos. Nuestro objetivo será transformar ese _dataset_ 2D en uno 3D, donde uno de los ejes representará los días, otro represente la hora del día y, el tercer eje, las diferentes columnas de datos (tiempo, temperatura,...).

<br>

![](data/image-lect/bikeshare_dataset.png)

<br>


#### 3.1 Loading the data
---
Los datos se encuentrán en un CSV, cuya primera fila contiene los nombres de las diferentes columnas

In [1]:
import numpy as np
import torch

bikes_path = "data/tabular/hour-fixed.csv"

bikes_numpy = np.loadtxt(
    bikes_path,
    dtype=np.float32,
    delimiter=",",
    skiprows=1,
    converters={1: lambda x: float(x[8:10])} # extrae sólo el día del mes del campo fecha
)

bikes = torch.from_numpy(bikes_numpy)
bikes

tensor([[1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 3.0000e+00, 1.3000e+01,
         1.6000e+01],
        [2.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 8.0000e+00, 3.2000e+01,
         4.0000e+01],
        [3.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 5.0000e+00, 2.7000e+01,
         3.2000e+01],
        ...,
        [1.7377e+04, 3.1000e+01, 1.0000e+00,  ..., 7.0000e+00, 8.3000e+01,
         9.0000e+01],
        [1.7378e+04, 3.1000e+01, 1.0000e+00,  ..., 1.3000e+01, 4.8000e+01,
         6.1000e+01],
        [1.7379e+04, 3.1000e+01, 1.0000e+00,  ..., 1.2000e+01, 3.7000e+01,
         4.9000e+01]])

Los nombres de las columnas de datos son los siguientes:

- 'instant', índice del registro
- 'dteday', días del mes
- 'season', (1:primavera, 2:verano, 3:otoño, 4:invierno)
- 'yr', año (2011, 2012)
- 'mnth', mes \[1, 12]
- 'hr', hora \[0, 23]
- 'holiday', flag de vacaciones
- 'weekday', día de la semana
- 'workingday', flag de día laborable
- 'weathersit', (1:despejado, 2:llovizna, 3:lluvia/nieve ligera, 4: lluvia/nieve fuerte)
- 'temp', temp ºC
- 'atemp', temp percibida ºC
- 'hum', humedad
- 'windspeed', velocidad del viento
- 'casual', nº de usuarios casuales
- 'registered', nº de usuarios registrados
- 'cnt', total de alquileres

In [2]:
import csv

# column names
col_list = next(csv.reader(open(bikes_path), delimiter=','))
col_list

['instant',
 'dteday',
 'season',
 'yr',
 'mnth',
 'hr',
 'holiday',
 'weekday',
 'workingday',
 'weathersit',
 'temp',
 'atemp',
 'hum',
 'windspeed',
 'casual',
 'registered',
 'cnt']

En un _dataset_ de serie temporal como este, las filas representas _time-points_ sucesivos (están ordenadas por esa dimensión). La existencia de dicha ordenación nos brinda la oportunidad de explotar las relaciones temporales de los datos. Por ejemplo, nos permitiría predecir alquileres de bicicletas a una hora determinada basándonos en el hecho de que estaba lloviendo en las horas precedentes.

Ahora nos encontramos con la tarea de preparar los datos para que puedan ser procesados convenientemente por la red neuronal. Ésta necesita ver una secuencia de valores para cada variable, como número de alquileres, hora, temperatura. Es decir, N secuencias paralelas de tamaño C (canales, que vendrían siendo los campos del registro). Nuestra dimensión N representará el eje del tiempo, una entrada por hora en este caso.

Puede ser interesante reoordenar este _dataset_ de dos años en periodos de observación más amplios que una hora, como, por ejemplo, días. De esta forma, dispondremos de **N colecciones** de **C secuencias** de **longitud L**. La C se correspondería con nuestros 17 canales mientras que la longitud sería 24 (horas del día).

Vamos a obtener una nueva vista de nuestro _dataset_ para organizarlo en _batches_ de 24 horas. Echemos un vistazo a nuestro tensor:

In [3]:
bikes.shape

torch.Size([17520, 17])

Vamos a obtener una nueva vista de nuestro tensor para tener 3 ejes: día, hora y, luego, las 17 columnas. Para ello, usaremos el método _view()_ que recibe como argumento las dimensiones (_shape_) que tendrá el nuevo tensor.

In [4]:
daily_bikes = bikes.view(-1, 24, bikes.shape[1])
daily_bikes.shape, daily_bikes.stride()

(torch.Size([730, 24, 17]), (408, 17, 1))

La llamada al método _view()_ precedente recibe los siguientes argumentos: 730 (17520/24), 24 y 17. Para hacer la llamada al método de una forma más generalizada, pasamos directamente el tamaño de la dimensión 1 de bikes (17), fijamos la segunda dimensión a 24, y omitimos la primera de las dimensiones (empleando -1 como _placeholder_) de forma que PyTorch la tendrá que calcular automáticamente.

Vemos como según el nuevo _stride_, avanzar un día de datos supone desplazarse 408 (24 horas x 17 campos) en la estructura interna, y avanzar una hora supone "saltar" los 17 campos de la hora actual hasta posicionarse en la siguiente.

Finalmente, acabemos de reordenar nuestro tensor para que se ajuste al esquema deseado de: _N_x_C_x_L_. Esto podemos hacerlo de dos formas, empleando _transpose_ para hacer la transpuesta de las matrices internas (24 horas x 17 valores) o mediante _permute_ para intercambiar las dimensiones


In [6]:
#daily_bikes = daily_bikes.transpose(1,2)
daily_bikes = daily_bikes.permute(0,2,1)
daily_bikes.shape, daily_bikes.stride()

(torch.Size([730, 17, 24]), (408, 1, 17))

#### 3.2 Preparando el _dataset_ para el entrenamiento
---
La variable _"weather situation"_ es ordinal. Tiene cuatro posibles valores, desde **1** (buen tiempo) a **4** (mal tiempo). 

Por otro lado, podemos tratarla también como nominal o categórica. En este último caso, podríamos codificarla en un vector _one-hot_ y concatenar las nuevas columnas con el _dataset_.

Para simplificar nuestro experimento, vamos a limitarnos por el momento al primer día del _dataset_. Inicializaremos a zero una matriz con tantas filas como horas hay en el día (24) y tantas columnas como posibles situaciones contempladas del tiempo (4).

In [7]:
first_day = bikes[:24].long()
weather_onehot = torch.zeros(first_day.shape[0], 4)
first_day[:, 9] # weathersit column

tensor([1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 2, 2, 2, 2])

Ahora, deberemos rellenar nuestra matriz _one-hot_ a partir de esa columna de valores. Para ello, deberemos añadir una nueva dimensión al tensor con los datos del primer día para que ambos tensores tengan el mismo número de dimensiones (2).

In [9]:
weather_onehot.scatter_(
    dim=1, # dimension de la lista de valores que vamos a procesar
    index=first_day[:,9].unsqueeze(1).long() - 1, # valores - 1
    value=1.0 # valor que se insertará en cada posición 
)

tensor([[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.]])

Por último, concatenaremos este nuevo tensor al tensor _first_\__day_ empleando la función _cat()_

In [13]:
torch.cat((first_day, weather_onehot), 1) # concatena las columnas, la segunda dimension (1)

tensor([[  1.,   1.,   1.,   0.,   1.,   0.,   0.,   6.,   0.,   1.,   0.,   0.,
           0.,   0.,   3.,  13.,  16.,   1.,   0.,   0.,   0.],
        [  2.,   1.,   1.,   0.,   1.,   1.,   0.,   6.,   0.,   1.,   0.,   0.,
           0.,   0.,   8.,  32.,  40.,   1.,   0.,   0.,   0.],
        [  3.,   1.,   1.,   0.,   1.,   2.,   0.,   6.,   0.,   1.,   0.,   0.,
           0.,   0.,   5.,  27.,  32.,   1.,   0.,   0.,   0.],
        [  4.,   1.,   1.,   0.,   1.,   3.,   0.,   6.,   0.,   1.,   0.,   0.,
           0.,   0.,   3.,  10.,  13.,   1.,   0.,   0.,   0.],
        [  5.,   1.,   1.,   0.,   1.,   4.,   0.,   6.,   0.,   1.,   0.,   0.,
           0.,   0.,   0.,   1.,   1.,   1.,   0.,   0.,   0.],
        [  6.,   1.,   1.,   0.,   1.,   5.,   0.,   6.,   0.,   2.,   0.,   0.,
           0.,   0.,   0.,   1.,   1.,   0.,   1.,   0.,   0.],
        [  7.,   1.,   1.,   0.,   1.,   6.,   0.,   6.,   0.,   1.,   0.,   0.,
           0.,   0.,   2.,   0.,   2.,   1.,   0.

Tras nuestro pequeño experimento, vamos a hacer los mismo para todo el _dataset_ (daily_bikes). Recuerda que lo "remapeamos" como _NxCxL_, donde N representa cada uno de los días (730 = 2 años x 365 d), C son las variables (17) y L son las horas (24). 

Primero, crearemos un tensor del mismo N y L, pero de 4 canales

In [14]:
daily_weather_onehot = torch.zeros(daily_bikes.shape[0], 4, daily_bikes.shape[2])
daily_weather_onehot.shape

torch.Size([730, 4, 24])

Ahora hacemos el "scatter" de la columna "weathersit" sobre el nuevo tensor

In [18]:
daily_weather_onehot.scatter_(1, daily_bikes[:,9,:].long().unsqueeze(1) - 1, 1.0)
daily_weather_onehot.shape

torch.Size([730, 4, 24])

Y concatenamos a lo largo de la dimensión C:

In [20]:
daily_bikes = torch.cat((daily_bikes, daily_weather_onehot), dim=1)
daily_bikes.shape # la segunda dimensiónse incrementará en 4

torch.Size([730, 25, 24])